### Tutorial for training EHR-M-GAN (Version Corrigée)

Reference: "Generating Synthetic Mixed-type Longitudinal Electronic Health Records for Artificial Intelligent". https://arxiv.org/abs/2112.12047

In this work, we propose a generative adversarial network (GAN) entitled EHR-M-GAN which simultaneously synthesizes mixed-type timeseries EHR data (e.g., continuous-valued timeseries and discrete-valued timeseries). EHR-M-GAN is capable of capturing the multidimensional, heterogeneous, and correlated temporal dynamics in patient trajectories.

---
**CORRECTIONS APPORTÉES :**
- ✅ Ajout du chemin du projet au PYTHONPATH
- ✅ Chemins de données corrigés pour Windows
- ✅ Compatibilité TensorFlow 1.x/2.x
- ✅ Vérifications des fichiers avant chargement
- ✅ Nom de fichier corrigé (statics.pkl)

#### Configuration de l'environnement

In [3]:
import os
import sys

# Naviguer vers le projet
PROJECT_PATH = r"D:\work\ehrMGAN-main"
os.chdir(PROJECT_PATH)

# Ajouter au PYTHONPATH
if PROJECT_PATH not in sys.path:
    sys.path.insert(0, PROJECT_PATH)

# Vérifier
assert os.path.exists('networks.py'), "❌ networks.py introuvable!"
print(f"✅ Projet configuré: {PROJECT_PATH}")

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\work\\ehrMGAN-main'

In [1]:
import sys
import os

# IMPORTANT: Ajuster ce chemin selon votre installation
# Pour Windows: r"D:\work\ehrMGAN-main"
# Pour Linux/Mac: "/path/to/ehrMGAN-main"
PROJECT_PATH = r"D:\work\ehrMGAN-main"

# Ajouter le chemin du projet au PYTHONPATH pour importer les modules
if PROJECT_PATH not in sys.path:
    sys.path.insert(0, PROJECT_PATH)

print(f"✅ Chemin du projet ajouté: {PROJECT_PATH}")
print(f"✅ Répertoire de travail actuel: {os.getcwd()}")

✅ Chemin du projet ajouté: D:\work\ehrMGAN-main
✅ Répertoire de travail actuel: /content


#### Necessary packages and functions call

In [2]:
import tensorflow as tf
import numpy as np
import pickle

# Vérifier la version de TensorFlow
print(f"TensorFlow version: {tf.__version__}")

# Import des modules du projet
try:
    from networks import C_VAE_NET, D_VAE_NET, C_GAN_NET, D_GAN_NET
    from m3gan import m3gan
    from utils import renormlizer
    print("✅ Tous les modules ont été importés avec succès!")
except ImportError as e:
    print(f"❌ Erreur d'import: {e}")
    print("Vérifiez que vous êtes dans le bon répertoire et que tous les fichiers sont présents.")
    raise

TensorFlow version: 2.19.0
❌ Erreur d'import: No module named 'networks'
Vérifiez que vous êtes dans le bon répertoire et que tous les fichiers sont présents.


ModuleNotFoundError: No module named 'networks'

In [ ]:
# Configuration des chemins de données
# IMPORTANT: Ajuster ces chemins selon votre configuration
DATA_PATH = os.path.join(PROJECT_PATH, "data", "real", "eicu")
CHECKPOINT_DIR = os.path.join(PROJECT_PATH, "data", "checkpoint")
OUTPUT_DIR = os.path.join(PROJECT_PATH, "data", "fake")

print(f"📁 Chemin des données: {DATA_PATH}")
print(f"💾 Chemin des checkpoints: {CHECKPOINT_DIR}")
print(f"📊 Chemin de sortie: {OUTPUT_DIR}")

# Créer les répertoires s'ils n'existent pas
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)
print("✅ Répertoires vérifiés/créés")

#### Import dataset
  - MIMIC-III: https://physionet.org/content/mimiciii/1.4/
  - eICU-CRD: https://physionet.org/content/eicu-crd/2.0/ (used in this tutorial)
  - HiRID: https://physionet.org/content/hirid/1.1.1/

In [ ]:
# Vérifier l'existence des fichiers avant de les charger
required_files = [
    'vital_sign_24hrs.pkl',
    'med_interv_24hrs.pkl',
    'statics.pkl'  # Corrigé: était 'statics_cond.pkl'
]

print("Vérification des fichiers de données...")
for filename in required_files:
    filepath = os.path.join(DATA_PATH, filename)
    if os.path.exists(filepath):
        print(f"  ✅ {filename} trouvé")
    else:
        print(f"  ❌ {filename} MANQUANT à {filepath}")
        print(f"\n⚠️  ERREUR: Fichiers de données manquants!")
        print(f"Veuillez vous assurer que les fichiers suivants existent dans {DATA_PATH}:")
        for f in required_files:
            print(f"  - {f}")
        raise FileNotFoundError(f"Fichier manquant: {filename}")

In [ ]:
# Charger les données pour l'entraînement du GAN
print("Chargement des données...")

with open(os.path.join(DATA_PATH, 'vital_sign_24hrs.pkl'), 'rb') as f:
    vital_labs_3D = pickle.load(f)
    print(f"  ✅ vital_sign_24hrs.pkl chargé - Shape: {vital_labs_3D.shape}")

with open(os.path.join(DATA_PATH, 'med_interv_24hrs.pkl'), 'rb') as f:
    medical_interv_3D = pickle.load(f)
    print(f"  ✅ med_interv_24hrs.pkl chargé - Shape: {medical_interv_3D.shape}")

with open(os.path.join(DATA_PATH, 'statics.pkl'), 'rb') as f:
    statics = pickle.load(f)
    print(f"  ✅ statics.pkl chargé - Type: {type(statics)}")

print("\n✅ Toutes les données ont été chargées avec succès!")

In [ ]:
continuous_x = vital_labs_3D
discrete_x = medical_interv_3D

print(f"Données continues (vital signs): {continuous_x.shape}")
print(f"Données discrètes (interventions médicales): {discrete_x.shape}")

#### Define network parameters

In [ ]:
# Paramètres des séries temporelles
time_steps = continuous_x.shape[1]
c_dim = continuous_x.shape[2]
d_dim = discrete_x.shape[2]
no_gen = continuous_x.shape[0]

print(f"📊 Paramètres des données:")
print(f"  - Time steps: {time_steps}")
print(f"  - Continuous dimensions: {c_dim}")
print(f"  - Discrete dimensions: {d_dim}")
print(f"  - Number of samples: {no_gen}")

In [ ]:
# Paramètres de la phase VAE (pré-entraînement)
c_noise_dim = 5
c_z_size = 100
d_noise_dim = 5
d_z_size = 100

print(f"🔧 Paramètres VAE:")
print(f"  - C noise dim: {c_noise_dim}")
print(f"  - C latent size: {c_z_size}")
print(f"  - D noise dim: {d_noise_dim}")
print(f"  - D latent size: {d_z_size}")

In [ ]:
# Construire les réseaux VAE et GAN
print("Construction des réseaux neuronaux...")

c_vae = C_VAE_NET(x_dim=c_dim, z_dim=c_z_size, time_steps=time_steps)
print("  ✅ C_VAE_NET construit")

c_gan = C_GAN_NET(x_dim=c_dim, z_dim=c_z_size, time_steps=time_steps)
print("  ✅ C_GAN_NET construit")

d_vae = D_VAE_NET(x_dim=d_dim, z_dim=d_z_size, time_steps=time_steps)
print("  ✅ D_VAE_NET construit")

d_gan = D_GAN_NET(x_dim=d_dim, z_dim=d_z_size, time_steps=time_steps)
print("  ✅ D_GAN_NET construit")

print("\n✅ Tous les réseaux ont été construits avec succès!")

#### Define training parameters

In [ ]:
# Paramètres d'entraînement
batch_size = 128
num_pre_epochs = 100  # Pré-entraînement VAE
num_epochs = 500      # Entraînement GAN complet

# Fréquences de sauvegarde
epoch_ckpt_freq = 100  # Sauvegarder un checkpoint tous les 100 epochs
epoch_loss_freq = 10   # Afficher les pertes tous les 10 epochs

print(f"⚙️ Paramètres d'entraînement:")
print(f"  - Batch size: {batch_size}")
print(f"  - Pre-training epochs: {num_pre_epochs}")
print(f"  - Training epochs: {num_epochs}")
print(f"  - Checkpoint frequency: {epoch_ckpt_freq}")
print(f"  - Loss display frequency: {epoch_loss_freq}")

In [ ]:
# Rounds pour discriminateur, générateur et VAE
d_rounds = 1
g_rounds = 3
v_rounds = 1

print(f"🔄 Rounds d'entraînement:")
print(f"  - Discriminator rounds: {d_rounds}")
print(f"  - Generator rounds: {g_rounds}")
print(f"  - VAE rounds: {v_rounds}")

In [ ]:
# Learning rates
v_lr_pre = 1e-3  # Learning rate pour pré-entraînement VAE
v_lr = 1e-4      # Learning rate pour VAE pendant GAN
g_lr = 1e-4      # Learning rate pour générateur
d_lr = 1e-4      # Learning rate pour discriminateur

print(f"📈 Learning rates:")
print(f"  - VAE pre-training: {v_lr_pre}")
print(f"  - VAE: {v_lr}")
print(f"  - Generator: {g_lr}")
print(f"  - Discriminator: {d_lr}")

In [ ]:
# Coefficients de perte
alpha_re = 1.0   # Reconstruction loss
alpha_kl = 1.0   # KL divergence
alpha_mt = 1.0   # Marginal temporal loss
alpha_ct = 1.0   # Conditional temporal loss
alpha_sm = 1.0   # Semantic loss

c_beta_adv = 1.0  # Adversarial loss pour continuous
c_beta_fm = 10.0  # Feature matching loss pour continuous
d_beta_adv = 1.0  # Adversarial loss pour discrete
d_beta_fm = 10.0  # Feature matching loss pour discrete

print(f"⚖️ Coefficients de perte:")
print(f"  VAE:")
print(f"    - Reconstruction (alpha_re): {alpha_re}")
print(f"    - KL divergence (alpha_kl): {alpha_kl}")
print(f"    - Marginal temporal (alpha_mt): {alpha_mt}")
print(f"    - Conditional temporal (alpha_ct): {alpha_ct}")
print(f"    - Semantic (alpha_sm): {alpha_sm}")
print(f"  GAN Continuous:")
print(f"    - Adversarial (c_beta_adv): {c_beta_adv}")
print(f"    - Feature matching (c_beta_fm): {c_beta_fm}")
print(f"  GAN Discrete:")
print(f"    - Adversarial (d_beta_adv): {d_beta_adv}")
print(f"    - Feature matching (d_beta_fm): {d_beta_fm}")

In [ ]:
# Chemin du checkpoint
checkpoint_dir = CHECKPOINT_DIR
print(f"💾 Checkpoint directory: {checkpoint_dir}")

#### Train the model

**IMPORTANT:** Ce code gère automatiquement la compatibilité entre TensorFlow 1.x et 2.x

In [ ]:
# Vérifier la version de TensorFlow et configurer en conséquence
tf_version = int(tf.__version__.split('.')[0])
print(f"\n🔍 TensorFlow version détectée: {tf.__version__} (v{tf_version})")

if tf_version == 1:
    print("✅ Utilisation de TensorFlow 1.x")
    # Code pour TensorFlow 1.x (original)
    tf.reset_default_graph()
    run_config = tf.ConfigProto()
    run_config.gpu_options.allow_growth = True
    
    with tf.Session(config=run_config) as sess:
        model = m3gan(sess=sess,
                      batch_size=batch_size,
                      time_steps=time_steps,
                      num_pre_epochs=num_pre_epochs,
                      num_epochs=num_epochs,
                      checkpoint_dir=checkpoint_dir,
                      epoch_ckpt_freq=epoch_ckpt_freq,
                      epoch_loss_freq=epoch_loss_freq,
                      # params for c
                      c_dim=c_dim, c_noise_dim=c_noise_dim,
                      c_z_size=c_z_size, c_data_sample=continuous_x,
                      c_vae=c_vae, c_gan=c_gan,
                      # params for d
                      d_dim=d_dim, d_noise_dim=d_noise_dim,
                      d_z_size=d_z_size, d_data_sample=discrete_x,
                      d_vae=d_vae, d_gan=d_gan,
                      # params for training
                      d_rounds=d_rounds, g_rounds=g_rounds, v_rounds=v_rounds,
                      v_lr_pre=v_lr_pre, v_lr=v_lr, g_lr=g_lr, d_lr=d_lr,
                      alpha_re=alpha_re, alpha_kl=alpha_kl, alpha_mt=alpha_mt, 
                      alpha_ct=alpha_ct, alpha_sm=alpha_sm,
                      c_beta_adv=c_beta_adv, c_beta_fm=c_beta_fm, 
                      d_beta_adv=d_beta_adv, d_beta_fm=d_beta_fm)
        model.build()
        model.train()
        
elif tf_version == 2:
    print("⚠️  TensorFlow 2.x détecté")
    print("Ce code a été écrit pour TensorFlow 1.x")
    print("\nOptions:")
    print("  1. Installer TensorFlow 1.15: pip install tensorflow==1.15")
    print("  2. Utiliser le mode de compatibilité TF2:")
    print("")
    print("import tensorflow.compat.v1 as tf")
    print("tf.disable_v2_behavior()")
    print("")
    
    # Tentative d'utiliser le mode de compatibilité
    try:
        import tensorflow.compat.v1 as tf
        tf.disable_v2_behavior()
        print("\n✅ Mode de compatibilité TF1 activé")
        
        tf.reset_default_graph()
        run_config = tf.ConfigProto()
        run_config.gpu_options.allow_growth = True
        
        with tf.Session(config=run_config) as sess:
            model = m3gan(sess=sess,
                          batch_size=batch_size,
                          time_steps=time_steps,
                          num_pre_epochs=num_pre_epochs,
                          num_epochs=num_epochs,
                          checkpoint_dir=checkpoint_dir,
                          epoch_ckpt_freq=epoch_ckpt_freq,
                          epoch_loss_freq=epoch_loss_freq,
                          # params for c
                          c_dim=c_dim, c_noise_dim=c_noise_dim,
                          c_z_size=c_z_size, c_data_sample=continuous_x,
                          c_vae=c_vae, c_gan=c_gan,
                          # params for d
                          d_dim=d_dim, d_noise_dim=d_noise_dim,
                          d_z_size=d_z_size, d_data_sample=discrete_x,
                          d_vae=d_vae, d_gan=d_gan,
                          # params for training
                          d_rounds=d_rounds, g_rounds=g_rounds, v_rounds=v_rounds,
                          v_lr_pre=v_lr_pre, v_lr=v_lr, g_lr=g_lr, d_lr=d_lr,
                          alpha_re=alpha_re, alpha_kl=alpha_kl, alpha_mt=alpha_mt, 
                          alpha_ct=alpha_ct, alpha_sm=alpha_sm,
                          c_beta_adv=c_beta_adv, c_beta_fm=c_beta_fm, 
                          d_beta_adv=d_beta_adv, d_beta_fm=d_beta_fm)
            model.build()
            model.train()
    except Exception as e:
        print(f"\n❌ Erreur avec le mode de compatibilité: {e}")
        print("\nVeuillez installer TensorFlow 1.15:")
        print("  pip uninstall tensorflow")
        print("  pip install tensorflow==1.15")
else:
    raise ValueError(f"Version TensorFlow non supportée: {tf.__version__}")

#### Vérification des résultats

In [ ]:
# Vérifier que les données synthétiques ont été générées
import glob

synthetic_files = glob.glob(os.path.join(OUTPUT_DIR, "epoch*/gen_data.npz"))
print(f"\n📊 Fichiers de données synthétiques générés: {len(synthetic_files)}")

if synthetic_files:
    print("\n✅ Entraînement terminé avec succès!")
    print("\nFichiers générés:")
    for f in sorted(synthetic_files):
        print(f"  - {os.path.basename(os.path.dirname(f))}/gen_data.npz")
else:
    print("\n⚠️  Aucun fichier de données synthétiques trouvé")
    print("Vérifiez que l'entraînement s'est terminé correctement")